In [28]:
#Import packages

import numpy as np
import pandas as pd
from PIL import Image
import os
import path

import torch
import torchvision
from torch.utils import data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [29]:
def target_to_oh(target):
    NUM_CLASS = 5  # hard code here, can do partial
    one_hot = torch.eye(NUM_CLASS)[target]
    return one_hot

In [30]:
transform_bunch = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.ToTensor()])
#                                       transforms.Normalize((0.5139, 0.2727, 0.0891), (0.1533, 0.0909, 0.0400))])

In [31]:
ds = torchvision.datasets.ImageFolder("../../gaussian_filtered_images/gaussian_filtered_images", transform_bunch, target_transform = target_to_oh)

In [32]:
for x,y in ds:
    print(y)
    break

tensor([1., 0., 0., 0., 0.])


In [33]:
percent = 0.2

split = int(len(ds) * percent)

indices = list(range(len(ds)))

In [34]:
split

732

In [35]:
train_sampler = SubsetRandomSampler(indices[split:])

valid_sampler = SubsetRandomSampler(indices[:split])

In [36]:
bs = 100

dl = data.DataLoader(ds, bs, sampler=train_sampler)

valid_dl = data.DataLoader(ds, bs, sampler=valid_sampler)

In [37]:
xx = 0
yy = 0

for x,y in valid_dl:
    print(x.shape,y.shape)
    xx = x
    yy = y
    break

torch.Size([100, 3, 224, 224]) torch.Size([100, 5])


In [38]:
yy

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0

In [39]:
#looks like something is off... Yup --> We can just transform it at the end
ds.class_to_idx

{'Mild': 0, 'Moderate': 1, 'No_DR': 2, 'Proliferate_DR': 3, 'Severe': 4}

In [40]:
#Now I'll be making the model! It'll take elements off of nn.Module
class cnnmaker(nn.Module):
    #Initialization will only take in input channels
    def __init__(self, input_channels):
        super().__init__()
        
        #I looked at what the resnet architechture looked like and just implmented the block types (not the residual part just yet)
        def block(in_chan):
            return nn.Sequential(nn.Conv2d(in_chan, in_chan * 2, 3, 2, 1, padding_mode = "reflect"),
                                nn.BatchNorm2d(in_chan*2),
                                nn.ReLU())
        
        self.model = nn.Sequential(block(input_channels),
                                  block(input_channels*2),
                                  block(input_channels*4),
                                  block(input_channels*8),
                                  block(input_channels*16))
        
        self.second_model = nn.Sequential(nn.Linear(4704, 500),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(500, 100),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(100, 5))
    #The forward pass when you call it   
    def forward(self, images):
        pre_proc = self.model(images)
#         print(pre_proc.shape)
        
        formatted = torch.reshape(pre_proc, (images.shape[0], -1))
        
        return self.second_model(formatted)

In [41]:
model = cnnmaker(3)

In [42]:
model

cnnmaker(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(12, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): Sequential(
      (0):

In [43]:
#Look at how many parameters
total = 0
for param in model.parameters():
    if param.requires_grad:
        total += param.numel()

print(total)

2458905


In [44]:
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [45]:
# model = model.to('cuda')
# loss_func = loss_func.to('cuda')

In [46]:
def accuracy(preds, target):
    correct = (preds == target).float()
    accuracy = correct.sum() / len(correct)
    return accuracy

In [47]:
# def train():
#     model.train()
    
#     for image, label in dl:
#         image, label = image.type(dtype=torch.cuda.FloatTensor), label.type(dtype=torch.cuda.FloatTensor)
#         optimizer.zero_grad()
#         prediction = model(image)
        
#         loss = loss_func(prediction, label)
        
#         loss.backward()
        
#         optimizer.step()
    
#     with torch.no_grad():
#         total_iter = 0
#         total_acc = 0
#         for image, label in valid_dl: 
#             image, label = image.to('cuda', dtype=torch.float), label.to('cuda', dtype=torch.float)
#             prediction = model(image)
#             total_acc += accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
#             total_iter += 1
#             if total_iter == 7:
#                 print("valid",total_acc/total_iter)
#                 total_iter = 0
#                 total_acc = 0
#                 for image, label in dl: 
#                     image, label = image.to('cuda', dtype=torch.float), label.to('cuda', dtype=torch.float)
#                     prediction = model(image)
#                     total_acc += accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
#                     total_iter += 1
#                     if total_iter == 7:
#                         print(total_acc/total_iter)
#                         return "done"

In [48]:
def train():
    model.train()
    
    for image, label in dl:
        # Memastikan tensor berada di perangkat CPU
        image, label = image.type(dtype=torch.FloatTensor), label.type(dtype=torch.FloatTensor)
        optimizer.zero_grad()
        prediction = model(image)
        
        loss = loss_func(prediction, label)
        
        loss.backward()
        
        optimizer.step()
    
    with torch.no_grad():
        total_iter = 0
        total_acc = 0
        for image, label in valid_dl: 
            # Memastikan tensor berada di perangkat CPU
            image, label = image.to('cpu', dtype=torch.float), label.to('cpu', dtype=torch.float)
            prediction = model(image)
            total_acc += accuracy(torch.argmax(prediction, dim=1).float(), torch.argmax(label, dim=1).float())
            total_iter += 1
            if total_iter == 7:
                print("valid", total_acc / total_iter)
                total_iter = 0
                total_acc = 0
                for image, label in dl: 
                    # Memastikan tensor berada di perangkat CPU
                    image, label = image.to('cpu', dtype=torch.float), label.to('cpu', dtype=torch.float)
                    prediction = model(image)
                    total_acc += accuracy(torch.argmax(prediction, dim=1).float(), torch.argmax(label, dim=1).float())
                    total_iter += 1
                    if total_iter == 7:
                        print(total_acc / total_iter)
                        return "done"


In [49]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.003)

In [50]:
for _ in range(4):
    train()

valid tensor(0.2543)
tensor(0.7571)
valid tensor(0.3329)
tensor(0.7743)
valid tensor(0.3371)
tensor(0.7700)
valid tensor(0.3343)
tensor(0.8029)


In [51]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [52]:
for _ in range(10):
    train()

valid tensor(0.3743)
tensor(0.7843)
valid tensor(0.3586)
tensor(0.8129)
valid tensor(0.4071)
tensor(0.7943)
valid tensor(0.4186)
tensor(0.7843)
valid tensor(0.3929)
tensor(0.7714)
valid tensor(0.3743)
tensor(0.7986)
valid tensor(0.3843)
tensor(0.8171)
valid tensor(0.3229)
tensor(0.8014)
valid tensor(0.3686)
tensor(0.7886)
valid tensor(0.4029)
tensor(0.7986)


In [53]:
for _ in range(20):
    train()

valid tensor(0.3929)
tensor(0.7814)
valid tensor(0.3814)
tensor(0.7971)
valid tensor(0.4129)
tensor(0.8186)
valid tensor(0.4386)
tensor(0.8114)
valid tensor(0.3857)
tensor(0.8014)
valid tensor(0.3686)
tensor(0.8029)
valid tensor(0.3986)
tensor(0.8014)
valid tensor(0.3771)
tensor(0.8100)
valid tensor(0.4057)
tensor(0.8129)
valid tensor(0.3657)
tensor(0.7986)
valid tensor(0.3971)
tensor(0.8243)
valid tensor(0.3700)
tensor(0.8057)
valid tensor(0.3571)
tensor(0.8286)
valid tensor(0.3771)
tensor(0.8371)
valid tensor(0.3900)
tensor(0.8171)
valid tensor(0.3671)
tensor(0.8186)
valid tensor(0.4214)
tensor(0.8500)
valid tensor(0.3729)
tensor(0.8486)
valid tensor(0.4100)
tensor(0.8129)
valid tensor(0.4057)
tensor(0.8386)


In [ ]:
for _ in range(20):
    train()

valid tensor(0.4029)
tensor(0.8257)
valid tensor(0.3514)
tensor(0.8414)
valid tensor(0.4300)
tensor(0.8086)
valid tensor(0.3714)
tensor(0.8571)
valid tensor(0.4157)
tensor(0.8514)
valid tensor(0.4029)
tensor(0.8486)
valid tensor(0.3743)
tensor(0.8457)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
for _ in range(20):
    train()

In [ ]:
xxx = 0
yyy = 0

for x,y in dl:
    print(x.shape,y.shape)
    xxx = x
    yyy = y
    break

In [ ]:
yyy = yyy.type(dtype=torch.FloatTensor)
torch.argmax(yyy, dim = 1).float()

In [ ]:
xxx = xxx.type(dtype=torch.FloatTensor)
prediction = model(xxx)
torch.argmax(prediction, dim = 1).float()


In [ ]:
accuracy(torch.argmax(prediction, dim = 1).float(),torch.argmax(yyy, dim = 1).float())

In [ ]:
with torch.no_grad():
    total_iter = 0
    total_acc = 0
    for image, label in dl: 
        image, label = image.to('cpu', dtype=torch.float), label.to('cpu', dtype=torch.float)
        prediction = model(image)
        h = accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
        print(h)
        total_acc += h
        total_iter += 1
        if total_iter == 20:
            print(total_acc/total_iter)